In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb
from sklearn.metrics import accuracy_score

In [27]:
df = pd.read_csv('/content/drive/MyDrive/loan_data.csv')
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


## Tiền xử lý

In [28]:
for i in range(len(df)):
    df['Loan_ID'][i] = df['Loan_ID'][i][2:len('xx000000')]
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [29]:
area = df.filter(['Property_Area'])
area = pd.get_dummies(area)
area

,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1
...,...,...,...
609,1,0,0
610,1,0,0
611,0,0,1
612,0,0,1


In [30]:
df['Dependents'].unique()

array(['0', '1', '2', '3+', nan], dtype=object)

In [31]:
gender = df['Gender'].replace(['Female','Male'],[0,1])
married = df['Married'].replace(['No','Yes'],[0,1])
education = df['Education'].replace(['Not Graduate','Graduate'],[0,1])

df = df.dropna()
df.loc[df['Dependents'] == '3+', 'Dependents'] = 3
# for i in range(len(df)):
#     df['Dependents'][i] = 3 if df['Dependents'][i] == '3+' else df['Dependents'][i]

self_employed = df['Self_Employed'].replace(['No','Yes'],[0,1])
loan_status = df['Loan_Status'].replace(['N','Y'],[0,1])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [32]:
df_new = df.drop(columns=['Property_Area', 'Gender', 'Married', 'Education', 'Self_Employed', 'Loan_Status'])
df_new = df_new.join(gender)
df_new = df_new.join(married)
df_new = df_new.join(education)
df_new = df_new.join(self_employed)
df_new = df_new.join(loan_status)
df_new = df_new.join(area)
df_new

,Loan_ID,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender,Married,Education,Self_Employed,Loan_Status,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
1,001003,1,4583,1508.0,128.0,360.0,1.0,1.0,1.0,1,0,0,1,0,0
2,001005,0,3000,0.0,66.0,360.0,1.0,1.0,1.0,1,1,1,0,0,1
3,001006,0,2583,2358.0,120.0,360.0,1.0,1.0,1.0,0,0,1,0,0,1
4,001008,0,6000,0.0,141.0,360.0,1.0,1.0,0.0,1,0,1,0,0,1
5,001011,2,5417,4196.0,267.0,360.0,1.0,1.0,1.0,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,002978,0,2900,0.0,71.0,360.0,1.0,0.0,0.0,1,0,1,1,0,0
610,002979,3,4106,0.0,40.0,180.0,1.0,1.0,1.0,1,0,1,1,0,0
611,002983,1,8072,240.0,253.0,360.0,1.0,1.0,1.0,1,0,1,0,0,1
612,002984,2,7583,0.0,187.0,360.0,1.0,1.0,1.0,1,0,1,0,0,1


In [33]:
df_new = df_new.astype('float')

In [34]:
X = df_new.drop(columns=['Loan_Status'])
y = df_new['Loan_Status']

In [35]:
X = StandardScaler().fit_transform(X)

## Gradient Boost

In [36]:
cv = KFold(n_splits=5, random_state=1, shuffle=True)
model = GBC(n_estimators=500)
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print(f"Accuracy score of Gradient Boost with K-Fold: {np.mean(scores)}\nStandard deviation: {np.std(scores)}")

Accuracy score of Gradient Boost with K-Fold: 0.7604166666666667
Standard deviation: 0.04166666666666667


## XGBoost

In [37]:
cv = KFold(n_splits=5, random_state=1, shuffle=True)
model = xgb.XGBClassifier(objective="binary:logistic")
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print(f"Accuracy score of XGBoost with K-Fold: {np.mean(scores)}\nStandard deviation: {np.std(scores)}")

Accuracy score of XGBoost with K-Fold: 0.7770833333333333
Standard deviation: 0.05877806658194121
